In [1]:
filename='testtext.txt'

In [2]:
with open(filename) as f:
    textstring=f.read()

In [4]:
import nltk.data

In [6]:
sent_detector=nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
print('\n-----\n'.join(sent_detector.tokenize(textstring.strip())))

Because of smartphone apps like Uber, taxi politics have gotten a lot of press lately.
-----
One of the claims Uber proponents make is that normal taxis do not serve everybody.
-----
In New York City, taxis are heavily regulated.
-----
There are only about 14,000 taxis allowed to pick up passengers off the street in the center of the city.
-----
Uber gets around this regulation by only picking up passengers who request a ride on the app.
-----
Because there are few taxis, critics say that they can get away with only serving the few.
-----
Some claim that taxis only serve those with the privilege to live in the city center, and that they are biased against people of color.
-----
As a New Yorker who has spent my fair share of time waiting in taxi traffic on my bicycle, I wanted to get some understanding about this issue.
-----
Are taxis actually unfair, as critics claim?


In [16]:
sent_list=sent_detector.tokenize(textstring.strip(),realign_boundaries=False)

In [24]:
import string

In [ ]:
def replace_dashes(st):
    

In [47]:
def remove_stuff(st,remove):
    import string
    return st.translate(string.maketrans('',''),remove)

In [48]:
def remove_all(st):
    return remove_stuff(st,string.digits+string.punctuation)

In [49]:
def process_string(st):
    return remove_all(st.replace('-',' ')).lower().split()

In [50]:
process_string(sent_list[0])

['because',
 'of',
 'smartphone',
 'apps',
 'like',
 'uber',
 'taxi',
 'politics',
 'have',
 'gotten',
 'a',
 'lot',
 'of',
 'press',
 'lately']

In [54]:
words_sent_list=map(process_string,sent_list)

In [55]:
map(len,words_sent_list)

[15, 15, 8, 19, 17, 17, 25, 28, 7]

In [42]:
map(lambda x: remove_pukt(x).lower().split(),sent_list)

AttributeError: 'tuple' object has no attribute 'lower'

In [36]:
word_sent_list=map(lambda x: remove_pukt(x).lower().split(),sent_list)

In [37]:
map(len,word_sent_list)

[15, 15, 8, 20, 17, 17, 25, 28, 7]

In [61]:
params=[ 4.60009789, -0.8802993 ]
import numpy as np
def equiv_sylb(freq,params):
    return np.exp((np.log(freq)-params[0])/params[1])

In [62]:
equiv_sylb(1,params)

185.97396035548638

In [63]:
import pandas as pd

In [66]:
freq_ser=pd.read_pickle('brown_frequency.pkl')

In [72]:
just_words=[item for sublist in words_sent_list for item in sublist]

In [86]:
frequency=np.array(map(lambda x: get_frequency(x,freq_ser),just_words))

In [87]:
equiv_sylb(frequency,params)

array([  3.24448815e-01,   6.42846044e-03,   1.85973960e+02,
         1.85973960e+02,   1.06083361e-01,   1.85973960e+02,
         1.75211431e+01,   7.44200790e+00,   5.88194379e-02,
         1.22026376e+01,   7.52718818e-03,   1.65071742e+00,
         6.42846044e-03,   8.57904994e+00,   3.85057575e+01,
         7.66925002e-02,   6.42846044e-03,   2.71780376e-03,
         2.98839567e+01,   1.85973960e+02,   1.85973960e+02,
         3.17706964e-01,   3.93132860e-02,   1.88646949e-02,
         5.02994845e+00,   1.85973960e+02,   1.53138265e-01,
         4.74430609e-02,   6.97414704e+00,   3.07617203e+00,
         1.12661431e-02,   3.60937121e-01,   1.85973960e+02,
         1.96082879e+00,   1.85973960e+02,   1.14576456e-01,
         4.40002791e+00,   5.33891783e+01,   9.39383230e-02,
         1.14576456e-01,   1.48261113e-01,   9.79727730e-02,
         1.85973960e+02,   9.27847788e+00,   6.90787821e-03,
         3.90369575e+00,   7.11597850e-02,   3.85057575e+01,
         2.57024666e-01,

In [85]:
def get_frequency(word,ser):
    try:
        return ser.loc[word]
    except KeyError:
        return 1
